In [1]:
# -*- coding=utf-8 -*-

import matplotlib.pyplot as plt
import pydicom
import pydicom.uid
import sys
import PIL.Image as Image
from PyQt5 import QtGui
import os

读取Dicom图像数据与得到的CT值图像（CT图）

In [2]:
have_numpy = True
try:
    import numpy
except ImportError:
    have_numpy = False
    raise
have_numpy

True

In [4]:
sys_is_little_endian = (sys.byteorder == 'little')
sys_is_little_endian

True

In [6]:
NumpySupportedTransferSyntaxes = [
    pydicom.uid.ExplicitVRLittleEndian,
    pydicom.uid.ImplicitVRLittleEndian,
    pydicom.uid.DeflatedExplicitVRLittleEndian,
    pydicom.uid.ExplicitVRBigEndian,
]

# 支持的传输语法
def supports_transfer_syntaxz(dicom_dataset):
    """
    Returns
    - - - - - - 
    bool
        True ：如果这个像素数据处理程序支持这种传输语法。
        Fales：防止任何试图使用此处理程序解码给定传输语法的尝试
    """
    return (dicom_dataset.file_meta.TransferSyntaxUID in NumpySupportedTransferSyntaxes)

# 转换为RGB
def needs_to_convert_to_RGB(dicom_dataset):
    return False

# 转换光度计
def should_change_PhotometricInterpretation_to_RGB(dicom_dataset):
    return False

# 加载Dicom图像数据
def get_pixeldata(dicom_dataset):
    """
    如果Numpy可用，返回PixelData的ndarray。
    
    Raises
    - - - - - - - - -
    TypeError
            如果没有PixelData或不支持数据类型
    ImportError
            如果Numpy没有
    NotImplementedError
            如果不支持传输语法
    AttributeError
            如果解码的数据量与期望的数据量不匹配
    Returns
    - - - - - - - - -
    numpy.ndarray
            （7FE0,0010）的内容作为ndarray。
    """
    if(dicom_dataset.file_meta.TransferSyntaxUID not in NumpySupportedTransferSyntaxes):
        raise NotImplementedError("Pixel Data is compressed in a "
                                  "format pydicom does not yet handle. "
                                  "Cannot return array. Pydicom might "
                                  "be able to convert the pixel data "
                                  "using GDCM if it is installed.")
        
    if not have_numpy:
        msg  = ("The Numpy package is required to use pixel_array, and "
               "numpy could not be imported.")
        
        raise ImportError(msg)
    if "PixelData" not in dicom_dataset:
        raise TypeError("No pixel data found in this dataset.")
        
    # 制作NumPy的format code, e.g. "uint16", "int32"等等
    # from two pieces of info:
    # dicom_dataset.PixelRepresentation：0 for unsigned, 1 for signed;
    # dicom_dataset.BitsAllocated：8, 16, or 32
    if dicom_dataset.BitsAllocated == 1:
        # 单个bit用来表示二进制数据
        format_str = 'uint8'
    elif dicom_dataset.PixelRepresentation == 0:
        format_str = 'uint{}'.format(dicom_dataset.BitsAllocated)
    elif dicom_dataset.PixelRepresentation == 1:
        format_str = 'int{}'.format(dicom_dataset.BitsAllocated)
    else:
        format_str = 'bad_pixel_representation'
    try:
        numpy_dtype = numpy.dtype(format_str)
    except TypeError:
        msg = ("Data type not understood by NumPy: "
               "format='{}', PixelRepresentation={}, "
               "BitsAllocated={}".format(
                   format_str,
                   dicom_dataset.PixelRepresentation,
                   dicom_dataset.BitsAllocated))
        raise TypeError(msg)
        
    if dicom_dataset.is_little_endian != sys_is_little_endian:
        numpy_dtype = numpy_dtype.newbyteorder('S')
        
    # 获得图像的二进制值    
    pixel_bytearray = dicom_dataset.PixelData
    
    if dicom_dataset.BitsAllocated == 1:
        # if single bits are used for binary representation, a uint8 array
        # has to be converted to a binary-valued array (that is 8 times bigger)
        try:
            pixel_array = numpy.unpackbits(
                numpy.frombuffer(pixel_bytearray, dtype='uint8'))
        except NotImplementedError:
            # PyPy2 does not implement numpy.unpackbits
            raise NotImplementedError(
                'Cannot handle BitsAllocated == 1 on this platform')
    else:
        pixel_array = numpy.frombuffer(pixel_bytearray, dtype=numpy_dtype)
    
    # 获取数据长度：单位byte
    length_of_pixel_array = pixel_array.nbytes
    # 期望的长度：单位byte
    expected_length = dicom_dataset.Rows * dicom_dataset.Columns 
    
    if ('NumberOfFrames' in dicom_dataset and dicom_dataset.NumberOfFrames > 1):
        expected_length *= dicom_dataset.NumberOfFrames
    if ('SamplesPerPixel' in dicom_dataset and dicom_dataset.SamplesPerPixel > 1):
        expected_length *= dicom_dataset.SamplesPerPixel
    if dicom_dataset.BitsAllocated > 8:
        expected_length *= (dicom_dataset.BitsAllocated // 8)
        
    padded_length = expected_length
    
    # expected_length&1的值为1代表其为奇数，为0则代表其为偶数
    if expected_length & 1:
        padded_length += 1
        
    if length_of_pixel_array != padded_length:
        raise AttributeError(
            "Amount of pixel data %d does not "
            "match the expected data %d" %
            (length_of_pixel_array, padded_length))
        
    if expected_length != padded_length:
        pixel_array = pixel_array[:expected_length]
        
    if should_change_PhotometricInterpretation_to_RGB(dicom_dataset):
        dicom_dataset.PhotometricInterpretation = "RGB"
        
    # CT图像需要得到其CT值图像，pixel_array转变为ct值
    if dicom_dataset.Modality.lower().find('ct') >= 0: 
        # 获得图像的CT值
        pixel_array = pixel_array * dicom_dataset.RescaleSlope + dicom_dataset.RescaleIntercept  
   
    pixel_array = pixel_array.reshape(dicom_dataset.Rows, dicom_dataset.Columns*dicom_dataset.SamplesPerPixel)
    return pixel_array, dicom_dataset.Rows, dicom_dataset.Columns

Step2:对于CT图像设置窗宽窗位

In [7]:
# 调整CT图像的窗宽窗位
def setDicomWinWidthWinCenter(img_data, winwidth, wincenter, rows, cols):
    img_temp = img_data
    img_temp.flags.writeable = True
    min = (2 * wincenter - winwidth) / 2.0 + 0.5
    max = (2 * wincenter + winwidth) / 2.0 + 0.5
    
    # 变换因子，将数值映射为0-255之间的数字
    dFactor = 255.0 / (max-min)

    for i in numpy.arange(rows):
        for j in numpy.arange(cols):
            img_temp[i, j] = int((img_temp[i, j]-min)*dFactor)

    # 处理有效灰度阈外的数值        
    min_index = img_temp < 0
    img_temp[min_index] = 0
    max_index = img_temp > 255
    img_temp[max_index] = 255

    return img_temp

step3：获取Dicom中的tag信息

In [8]:
# 加载Dicom图片中的Tag信息
def loadFileInformation(filename):
    information = {}
    ds = pydicom.read_file(filename)
    information['PatientID'] = ds.PatientID
    information['PatientName'] = ds.PatientName
    information['PatientBirthDate'] = ds.PatientBirthDate
    information['PatientSex'] = ds.PatientSex
    information['StudyID'] = ds.StudyID
    information['StudyDate'] = ds.StudyDate
    information['StudyTime'] = ds.StudyTime
    information['InstitutionName'] = ds.InstitutionName
    information['Manufacturer'] = ds.Manufacturer
    # print(dir(ds))
    # print(type(information))
    return information

step4：dicom图像数据转换为PIL.Image

In [ ]:
dcm = pydicom.dcmread(fileanme)  # 加载Dicom数据
dcm_img = Image.fromarray(img_data)  # 将Numpy转换为PIL.Image
dcm_img = dcm_img.convert('L')

# 保存为jpg文件，用作后面的生成label用
dcm_img.save('temp.jpg')
# 显示图像
dcm_img.show()